In [56]:
import numpy as np

import pandas as pd
import sklearn
from sklearn import preprocessing

import bokeh
#from bokeh.charts import BoxPlot
import bokeh.plotting
from bokeh.plotting import ColumnDataSource
from bokeh.models import LabelSet
from bokeh.models import FuncTickFormatter
from bokeh.io import export_svgs

import bokeh.io
bokeh.io.output_notebook()

Loading BokehJS ...

In [57]:
#load each run into a separate dataframe
df1 = pd.read_csv('190926_Deuterium_Transfer_Peak_Areas.csv', comment='#')
df2 = pd.read_csv('191007_Deuterium_Transfer_Peak_Areas.csv', comment='#')
df3 = pd.read_csv('191010_Deuterium_Transfer_Peak_Areas.csv', comment='#')
df4 = pd.read_csv('191015_Deuterium_Transfer_Peak_Areas.csv', comment='#')
df5 = pd.read_csv('191020_Deuterium_Transfer_Peak_Areas.csv', comment='#')
df6 = pd.read_csv('191021_Deuterium_Transfer_Peak_Areas.csv', comment='#')
df7 = pd.read_csv('191022_Deuterium_Transfer_Peak_Areas.csv', comment='#')
df8 = pd.read_csv('191025_Deuterium_Transfer_Peak_Areas.csv', comment='#')
df9 = pd.read_csv('191027_Deuterium_Transfer_Peak_Areas.csv', comment='#')
df10 = pd.read_csv('191031_Deuterium_Transfer_Peak_Areas.csv', comment='#')
df11 = pd.read_csv('191106_Deuterium_Transfer_Peak_Areas.csv', comment='#')
df12 = pd.read_csv('191115_Deuterium_Transfer_Peak_Areas.csv', comment='#')
df13 = pd.read_csv('191119_Deuterium_Transfer_Peak_Areas.csv', comment='#')

In [58]:
#add a column to each dataframe listing the date
df1['date'] = '190926'
df2['date'] = '191007'
df3['date'] = '191010'
df4['date'] = '191015'
df5['date'] = '191020'
df6['date'] = '191021'
df7['date'] = '191022'
df8['date'] = '191025'
df9['date'] = '191027'
df10['date'] = '191031'
df11['date'] = '191106'
df12['date'] = '191115'
df13['date'] = '191119'

#merge data frames into one dataframe
df=df1.append(df2).append(df3).append(df4).append(df5).append(df6).append(df7).append(df8).append(df9).append(df10).append(df11).append(df12).append(df13)

df.tail()

,Species,WellNumber,Peak ID,Ret Time,Start Tm,End Tm,m/z,Area,Area Percent,Height,Height Percent,A/H,Type,Outlier,date
43,L,8,D50C24,9.480,9.475,9.525,TIC,1020,0.01,2002,0.02,0.51,U,N,191119
44,L,8,D62C30,12.380,12.335,12.380,TIC,34904,0.30,33043,0.34,1.06,U,N,191119
45,D,8,C18,7.439,7.405,7.500,TIC,11443903,99.74,9614537,99.71,1.19,U,N,191119
46,D,8,D50C24,9.525,9.475,9.525,TIC,3225,0.03,3986,0.04,0.81,U,N,191119
47,D,8,D62C30,12.380,12.335,12.380,TIC,26344,0.23,24262,0.25,1.09,U,N,191119


In [59]:
#open metdata, which contains information regarding interaction time 
#between ant and beetle
metadata = pd.read_csv('Movement_metadata.csv')

#Date sync contains the dates that the video and GCMS data were collected on
#These two dates are different for every trial
date_sync = pd.read_csv('DCHC_Date_sync.csv')
metadata.head(2)

,Unnamed: 0,well number,species,collection date,file path,length lower,length upper,touching percent
0,0,3,S,191025,./191025/191025_DCHC_transfer_experiment_S_wel...,NaN,110,0.777064
1,1,7,S,191025,./191025/191025_DCHC_transfer_experiment_S_wel...,NaN,110,0.809745


In [60]:
date_sync.head(2)

,Video Date,GCMS Date
0,190919,190926
1,191003,191007


In [61]:
#rename the date column in the metadata and main dataframe so that they
#can be merged with the date_sync dataframe and subsequently merged with
#eachother
metadata = metadata.rename(columns={'collection date':'Video Date','well number':'WellNumber'})
df = df.rename(columns={'date':'GCMS Date'})
len(df)

624

In [62]:
df.head()

,Species,WellNumber,Peak ID,Ret Time,Start Tm,End Tm,m/z,Area,Area Percent,Height,Height Percent,A/H,Type,Outlier,GCMS Date
0,L,1,C18,7.435,7.405,7.470,TIC,5589973,93.17,5956210,95.09,0.94,L,N,190926
1,L,1,D50C24,9.498,9.475,9.525,TIC,231950,3.87,222378,3.55,1.04,L,N,190926
2,L,1,D62C30,12.350,12.330,12.375,TIC,177959,2.97,85013,1.36,2.09,L,N,190926
3,S,1,C18,7.434,7.410,7.455,TIC,6037087,99.11,6123349,99.37,0.99,L,N,190926
4,S,1,D50C24,9.497,9.475,9.525,TIC,37683,0.62,31671,0.51,1.19,L,N,190926


In [63]:
df['GCMS Date']=df['GCMS Date'].astype(int)
df['WellNumber']=df['WellNumber'].astype(int)

In [65]:
df = df.merge(date_sync).merge(metadata[['WellNumber','Video Date','touching percent']])
df.head()

,Species,WellNumber,Peak ID,Ret Time,Start Tm,End Tm,m/z,Area,Area Percent,Height,Height Percent,A/H,Type,Outlier,GCMS Date,Video Date,touching percent
0,L,1,C18,7.435,7.405,7.470,TIC,5589973,93.17,5956210,95.09,0.94,L,N,190926,190919,0.597702
1,L,1,D50C24,9.498,9.475,9.525,TIC,231950,3.87,222378,3.55,1.04,L,N,190926,190919,0.597702
2,L,1,D62C30,12.350,12.330,12.375,TIC,177959,2.97,85013,1.36,2.09,L,N,190926,190919,0.597702
3,S,1,C18,7.434,7.410,7.455,TIC,6037087,99.11,6123349,99.37,0.99,L,N,190926,190919,0.597702
4,S,1,D50C24,9.497,9.475,9.525,TIC,37683,0.62,31671,0.51,1.19,L,N,190926,190919,0.597702


In [66]:
len(df)

606

In [67]:
#function to normalize the peak are of each unique peak using the area of the C18 peak for each sample
def normalize_area(data):
    #check to make sure that the dataframe hasn't already been normalized
    if not {'Normalized Area', 'Hydrocarbon amount'}.issubset(data.columns):
        #Create array of c18 values repeated in groups of three, matching the number of unique peaks for each sample
        c18_Areas = np.repeat(data['Area'].loc[data['Peak ID'] == 'C18'],3).reset_index().drop("index", axis=1)
        
        #Divide the area of each peak in the dataframe by the corresponding c18 peak area
        data['Normalized Area'] = np.divide(data['Area'],c18_Areas['Area'])
        
        #Calculate the hydrocarbon amount, in ng, by multiplying the normalized peak area by 25.
        #25 ng of c18 was injected in each sample (1 microliter of a 25 ng/microliter solution of c18 in hexane)
        data['Hydrocarbon amount'] = data['Normalized Area']*25
        
    return data

In [68]:
df = normalize_area(df)

In [69]:
#Only scale the beetle values by the interaction percent. The ant, in interacting
#with itself has an effective interaction percent of 100
df['Scaled hydrocarbon amount'] = df['Hydrocarbon amount']
df['Scaled hydrocarbon amount'].loc[df['Species'] != 'L'] = df['Hydrocarbon amount'].loc[df['Species'] != 'L'] / df['touching percent'].loc[df['Species'] != 'L']
len(df)

/Users/TomNaragon/anaconda3/lib/python3.7/site-packages/pandas/core/indexing.py:190: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)


606

In [70]:
#function to create an identifier for each sample in the order (Liometopum or not : Beetle: Type)
def add_identifier(data):
    #check if identifier column already exists
    if not {'Identifier'}.issubset(data.columns):
        
        #create identifier column, populating it with the beetle species used in each well
        data = data.merge(data[['WellNumber','Species','GCMS Date']].loc[data['Species'] != 'L'].rename(columns={"Species": "Identifier"}).drop_duplicates())
        
        #modify the identifier column, adding a leading 'L' for Liometopum runs and adding a suffix from the 'Type' column
        data['Identifier'] = ((data['Species']=='L')*pd.Series('L',index=data.index)) + data['Identifier'] + data['Type']
        
    return data

In [71]:
df = add_identifier(df)

In [78]:
df = df.loc[df['Outlier']!='Y']
#df = df.loc[df['Outlier'] == 'N']
len(df)

576

In [79]:
np.unique(df['Outlier'])

array(['A', 'AI', 'B', 'BI', 'I', 'N'], dtype=object)

In [80]:
def box_and_whisker(data,plt,x_vals,y_vals):
    
    groups = data.groupby(x_vals)
    q1 = groups.quantile(q=0.25)
    q2 = groups.quantile(q=0.5)
    q3 = groups.quantile(q=0.75)
    iqr = q3 - q1
    upper = q3 + 1.5*iqr
    lower = q1 - 1.5*iqr

    # find the outliers for each category
    def outliers(group):
        cat = group.name
        return group[(group[y_vals] > upper.loc[cat][y_vals]) | (group[y_vals] < lower.loc[cat][y_vals])][y_vals]
    out = groups.apply(outliers).dropna()

    # prepare outlier data for plotting, we need coordinates for every outlier.
    if not out.empty:
        outx = []
        outy = []
        for keys in out.index:
            outx.append(keys[0])
            outy.append(out.loc[keys[0]].loc[keys[1]])

    # if no outliers, shrink lengths of stems to be no longer than the minimums or maximums
    qmin = groups.quantile(q=0.00)
    qmax = groups.quantile(q=1.00)
    upper[y_vals] = [min([x,y]) for (x,y) in zip(list(qmax.loc[:,y_vals]),upper[y_vals])]
    lower[y_vals] = [max([x,y]) for (x,y) in zip(list(qmin.loc[:,y_vals]),lower[y_vals])]

    # stems
    plt.segment(q3.reset_index()[x_vals], upper[y_vals], q3.reset_index()[x_vals], q3[y_vals], line_color="black")
    plt.segment(q3.reset_index()[x_vals], lower[y_vals], q3.reset_index()[x_vals], q1[y_vals], line_color="black")

    #boxes
    plt.vbar(q3.reset_index()[x_vals], 0.7, q2[y_vals], q3[y_vals], fill_color=None, line_color="black")
    plt.vbar(q3.reset_index()[x_vals], 0.7, q1[y_vals], q2[y_vals], fill_color=None, line_color="black")

    # whiskers (almost-0 height rects simpler than segments)
    plt.vbar(q3.reset_index()[x_vals], top=lower[y_vals],bottom=lower[y_vals], width=0.2, line_color="black")
    plt.vbar(q3.reset_index()[x_vals], top=upper[y_vals],bottom=upper[y_vals], width=0.2, line_color="black")
    
    return plt

In [81]:
df_C24 = df.loc[df['Peak ID'] == 'D50C24']
df_C30 = df.loc[df['Peak ID'] == 'D62C30']

In [82]:
plotting_df=df_C24

Identifiers = ['LSU','SU','LSL','SL','LPU','PU','LPL','PL','LDU','DU','LDL','DL',]
Labels={'LSU': '','SU': 'Sceptobius Control','LSL': '','SL': 'Sceptobius Treated',
        'LPU': '','PU': 'Platyusa Control','LPL': '','PL': 'Platyusa Treated',
        'LDU': '','DU': 'Dalotia Control','LDL': '','DL': 'Dalotia Treated'}

np.random.seed(666)  

p = bokeh.plotting.figure(plot_width=800,
                          plot_height=600,
                          title='C24',
                          x_range=Identifiers,
                          y_axis_label='ng D50C24',
                          y_axis_type='log')

colors=['#494949','#5F56FF','#494949','#5F56FF','#494949','#C42F2F','#494949','#C42F2F','#494949','#832161','#494949','#832161']

p = box_and_whisker(plotting_df,p,'Identifier','Scaled hydrocarbon amount')

for index, row in plotting_df[['WellNumber','GCMS Date']].drop_duplicates().iterrows():
        
    data = {'identifier': plotting_df['Identifier'].loc[(plotting_df['WellNumber']==row['WellNumber']) & (plotting_df['GCMS Date']==row['GCMS Date'])].values,
            'hydrocarbon amount': plotting_df['Scaled hydrocarbon amount'].loc[(plotting_df['WellNumber']==row['WellNumber']) & (plotting_df['GCMS Date']==row['GCMS Date'])].values}

    source = ColumnDataSource(data=data)
    outlier=np.unique(plotting_df['Outlier'].loc[(plotting_df['WellNumber']==row['WellNumber']) & (plotting_df['GCMS Date']==row['GCMS Date'])])

    offsetVal=(np.random.rand(1)[0]-0.5)*0.5
#     p.line(bokeh.transform.dodge('identifier',  offsetVal,  range=p.x_range),
#            'hydrocarbon amount',
#            source=source,
#            color='black',
#            alpha=0.3)

    p.circle(bokeh.transform.dodge('identifier',  offsetVal,  range=p.x_range),
           'hydrocarbon amount',
           source=source,
           color=bokeh.transform.factor_cmap('identifier',colors,Identifiers),
           alpha=0.6,
           size=7)
    
    if outlier=='AI':
        p.circle(bokeh.transform.dodge('identifier',  offsetVal,  range=p.x_range),
               'hydrocarbon amount',
               source=source,
               color='lime',
               alpha=1,
               size=7,
               legend='Ant Injured')
    if outlier=='A':
        p.circle(bokeh.transform.dodge('identifier',  offsetVal,  range=p.x_range),
               'hydrocarbon amount',
               source=source,
               color='purple',
               alpha=1,
               size=7,
               legend='Ant Chomped')
    if outlier=='BI':
        p.circle(bokeh.transform.dodge('identifier',  offsetVal,  range=p.x_range),
               'hydrocarbon amount',
               source=source,
               color='orange',
               alpha=1,
               size=7,
               legend='Beetle Injured')
    if outlier=='B':
        p.circle(bokeh.transform.dodge('identifier',  offsetVal,  range=p.x_range),
               'hydrocarbon amount',
               source=source,
               color='pink',
               alpha=1,
               size=7,
               legend='Beetle Chomped')
    if outlier=='Y':
        p.circle(bokeh.transform.dodge('identifier',  offsetVal,  range=p.x_range),
               'hydrocarbon amount',
               source=source,
               color='cyan',
               alpha=1,
               size=7,
               legend='Ant Crushed or Not Labeled')

p.xgrid.visible = False
p.ygrid.visible = False

# Add custom axis

p.xaxis.formatter = FuncTickFormatter(code="""
var labels = %s;
return labels[tick];
""" %Labels)

bokeh.io.show(p)

In [84]:
plotting_df=df_C30

Identifiers = ['LSU','SU','LSL','SL','LPU','PU','LPL','PL','LDU','DU','LDL','DL',]
Labels={'LSU': '','SU': 'Sceptobius Control','LSL': '','SL': 'Sceptobius Treated',
        'LPU': '','PU': 'Platyusa Control','LPL': '','PL': 'Platyusa Treated',
        'LDU': '','DU': 'Dalotia Control','LDL': '','DL': 'Dalotia Treated'}

np.random.seed(666)  

p = bokeh.plotting.figure(plot_width=800,
                          plot_height=600,
                          title='C30',
                          x_range=Identifiers,
                          y_axis_label='ng D62C30',
                          y_axis_type='log')

colors=['#494949','#5F56FF','#494949','#5F56FF','#494949','#C42F2F','#494949','#C42F2F','#494949','#832161','#494949','#832161']

p = box_and_whisker(plotting_df,p,'Identifier','Scaled hydrocarbon amount')

for index, row in plotting_df[['WellNumber','GCMS Date']].drop_duplicates().iterrows():
        
    data = {'identifier': plotting_df['Identifier'].loc[(plotting_df['WellNumber']==row['WellNumber']) & (plotting_df['GCMS Date']==row['GCMS Date'])].values,
            'hydrocarbon amount': plotting_df['Scaled hydrocarbon amount'].loc[(plotting_df['WellNumber']==row['WellNumber']) & (plotting_df['GCMS Date']==row['GCMS Date'])].values}

    source = ColumnDataSource(data=data)
    outlier=np.unique(plotting_df['Outlier'].loc[(plotting_df['WellNumber']==row['WellNumber']) & (plotting_df['GCMS Date']==row['GCMS Date'])])

    offsetVal=(np.random.rand(1)[0]-0.5)*0.5
#     p.line(bokeh.transform.dodge('identifier',  offsetVal,  range=p.x_range),
#            'hydrocarbon amount',
#            source=source,
#            color='black',
#            alpha=0.3)

    p.circle(bokeh.transform.dodge('identifier',  offsetVal,  range=p.x_range),
           'hydrocarbon amount',
           source=source,
           color=bokeh.transform.factor_cmap('identifier',colors,Identifiers),
           alpha=0.6,
           size=7)
    
    if outlier=='AI':
        p.circle(bokeh.transform.dodge('identifier',  offsetVal,  range=p.x_range),
               'hydrocarbon amount',
               source=source,
               color='lime',
               alpha=1,
               size=7,
               legend='Ant Injured')
    if outlier=='A':
        p.circle(bokeh.transform.dodge('identifier',  offsetVal,  range=p.x_range),
               'hydrocarbon amount',
               source=source,
               color='purple',
               alpha=1,
               size=7,
               legend='Ant Chomped')
    if outlier=='BI':
        p.circle(bokeh.transform.dodge('identifier',  offsetVal,  range=p.x_range),
               'hydrocarbon amount',
               source=source,
               color='orange',
               alpha=1,
               size=7,
               legend='Beetle Injured')
    if outlier=='B':
        p.circle(bokeh.transform.dodge('identifier',  offsetVal,  range=p.x_range),
               'hydrocarbon amount',
               source=source,
               color='pink',
               alpha=1,
               size=7,
               legend='Beetle Chomped')
    if outlier=='Y':
        p.circle(bokeh.transform.dodge('identifier',  offsetVal,  range=p.x_range),
               'hydrocarbon amount',
               source=source,
               color='cyan',
               alpha=1,
               size=7,
               legend='Ant Crushed or Not Labeled')

p.xgrid.visible = False
p.ygrid.visible = False

# Add custom axis

p.xaxis.formatter = FuncTickFormatter(code="""
var labels = %s;
return labels[tick];
""" %Labels)

bokeh.io.show(p)

In [86]:
df_C24['Scaled Amount']=0
for _,date in enumerate(np.unique(df_C24['GCMS Date'])):
    df_C24.loc[df_C24['GCMS Date']==date,'Scaled Amount'] = preprocessing.MinMaxScaler().fit_transform(df_C24.loc[df_C24['GCMS Date']==date,'Scaled hydrocarbon amount'].values.reshape(-1, 1)).ravel()

df_C30['Scaled Amount']=0
for _,date in enumerate(np.unique(df_C30['GCMS Date'])):
    df_C30.loc[df_C30['GCMS Date']==date,'Scaled Amount'] = preprocessing.MinMaxScaler().fit_transform(df_C30.loc[df_C30['GCMS Date']==date,'Scaled hydrocarbon amount'].values.reshape(-1, 1)).ravel()

/Users/TomNaragon/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.
/Users/TomNaragon/anaconda3/lib/python3.7/site-packages/pandas/core/indexing.py:543: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s
/Users/TomNaragon/anaconda3/lib/python3.7/site-packages/pandas/core/indexing.py:543: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = va

In [87]:
df_C24.head()

,Species,WellNumber,Peak ID,Ret Time,Start Tm,End Tm,m/z,Area,Area Percent,Height,...,Type,Outlier,GCMS Date,Video Date,touching percent,Normalized Area,Hydrocarbon amount,Scaled hydrocarbon amount,Identifier,Scaled Amount
1,L,1,D50C24,9.498,9.475,9.525,TIC,231950,3.87,222378,...,L,N,190926,190919,0.597702,0.041494,1.037348,1.037348,LSL,0.891385
4,S,1,D50C24,9.497,9.475,9.525,TIC,37683,0.62,31671,...,L,N,190926,190919,0.597702,0.006242,0.156048,0.261080,SL,0.220279
7,L,2,D50C24,9.525,9.475,9.525,TIC,1607,0.03,756,...,U,N,190926,190919,0.235601,0.000251,0.006282,0.006282,LSU,0.000000
10,S,2,D50C24,9.525,9.475,9.525,TIC,2344,0.04,1432,...,U,N,190926,190919,0.235601,0.000388,0.009708,0.041206,SU,0.030193
13,L,3,D50C24,9.525,9.475,9.525,TIC,4472,0.08,2979,...,U,BI,190926,190919,0.282222,0.000763,0.019070,0.019070,LSU,0.011056


In [90]:
plotting_df=df_C24

Identifiers = ['LSU','SU','LSL','SL','LPU','PU','LPL','PL','LDU','DU','LDL','DL',]
Labels={'LSU': '','SU': 'Sceptobius Control','LSL': '','SL': 'Sceptobius Treated',
        'LPU': '','PU': 'Platyusa Control','LPL': '','PL': 'Platyusa Treated',
        'LDU': '','DU': 'Dalotia Control','LDL': '','DL': 'Dalotia Treated'}

np.random.seed(666)  

p = bokeh.plotting.figure(plot_width=800,
                          plot_height=600,
                          title='C24',
                          x_range=Identifiers,
                          y_axis_label='scaled D50C24')

colors=['#494949','#5F56FF','#494949','#5F56FF','#494949','#C42F2F','#494949','#C42F2F','#494949','#832161','#494949','#832161']

for index, row in plotting_df[['WellNumber','GCMS Date']].drop_duplicates().iterrows():
        
    data = {'identifier': plotting_df['Identifier'].loc[(plotting_df['WellNumber']==row['WellNumber']) & (plotting_df['GCMS Date']==row['GCMS Date'])].values,
            'scaled amount': plotting_df['Scaled Amount'].loc[(plotting_df['WellNumber']==row['WellNumber']) & (plotting_df['GCMS Date']==row['GCMS Date'])].values}

    source = ColumnDataSource(data=data)

    offsetVal=(np.random.rand(1)[0]-0.5)*0.5

    p.circle(bokeh.transform.dodge('identifier',  offsetVal,  range=p.x_range),
               'scaled amount',
               source=source,
               color=bokeh.transform.factor_cmap('identifier',colors,Identifiers),
               alpha=0.6,
               size=7)


p.xgrid.visible = False
p.ygrid.visible = False

# Add custom axis

p.xaxis.formatter = FuncTickFormatter(code="""
var labels = %s;
return labels[tick];
""" %Labels)

# p.output_backend = "svg"
# export_svgs(p, filename="DCHC_transfer_C24_minmax_scaled.svg")

bokeh.io.show(p)

In [91]:
plotting_df=df_C30

Identifiers = ['LSU','SU','LSL','SL','LPU','PU','LPL','PL','LDU','DU','LDL','DL',]
Labels={'LSU': '','SU': 'Sceptobius Control','LSL': '','SL': 'Sceptobius Treated',
        'LPU': '','PU': 'Platyusa Control','LPL': '','PL': 'Platyusa Treated',
        'LDU': '','DU': 'Dalotia Control','LDL': '','DL': 'Dalotia Treated'}

np.random.seed(666)  

p = bokeh.plotting.figure(plot_width=800,
                          plot_height=600,
                          title='C30',
                          x_range=Identifiers,
                          y_axis_label='scaled D62C30')

colors=['#494949','#5F56FF','#494949','#5F56FF','#494949','#C42F2F','#494949','#C42F2F','#494949','#832161','#494949','#832161']

for index, row in plotting_df[['WellNumber','GCMS Date']].drop_duplicates().iterrows():
        
    data = {'identifier': plotting_df['Identifier'].loc[(plotting_df['WellNumber']==row['WellNumber']) & (plotting_df['GCMS Date']==row['GCMS Date'])].values,
            'scaled amount': plotting_df['Scaled Amount'].loc[(plotting_df['WellNumber']==row['WellNumber']) & (plotting_df['GCMS Date']==row['GCMS Date'])].values}

    source = ColumnDataSource(data=data)

    offsetVal=(np.random.rand(1)[0]-0.5)*0.5

    p.circle(bokeh.transform.dodge('identifier',  offsetVal,  range=p.x_range),
               'scaled amount',
               source=source,
               color=bokeh.transform.factor_cmap('identifier',colors,Identifiers),
               alpha=0.6,
               size=7)


p.xgrid.visible = False
p.ygrid.visible = False

# Add custom axis

p.xaxis.formatter = FuncTickFormatter(code="""
var labels = %s;
return labels[tick];
""" %Labels)

# p.output_backend = "svg"
# export_svgs(p, filename="DCHC_transfer_C24_minmax_scaled.svg")

bokeh.io.show(p)